# Make predictions using boltz2

In [7]:
#pip install boltz -U
# pip install PDBParser
import argparse
import json
import os
import shutil
import subprocess
import tempfile
from pathlib import Path
import pandas as pd
from Bio import PDB
from rdkit import Chem
import yaml
from Bio.PDB import PDBParser, PPBuilder

### First, we define a helper functions

- to extract protein seqeunce from availabe protein PDB file
- Generate yaml files for all 300 ligands
- Run boltz2 predictions for protein-ligand co-folding and affinity predictions inside a for loop

In [ ]:
def extract_protein_sequence(pdb_file):
    """
    Extracts the full protein sequence (one-letter code) from a PDB file.
    Returns the concatenated sequence as a string.
    """
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure("protein", pdb_file)
    ppb = PPBuilder()
    sequences = [pp.get_sequence() for pp in ppb.build_peptides(structure)]
    return "".join(str(seq) for seq in sequences)

def generate_yaml_files(csv_file, pdb_file, output_dir="yaml_files"):
    """
    Generates YAML files for each ligand in the CSV file for Boltz-2 input.
    """
    os.makedirs(output_dir, exist_ok=True)

    protein_sequence = extract_protein_sequence(pdb_file)
    print(f"Extracted protein sequence length: {len(protein_sequence)} residues")

    df = pd.read_csv(csv_file)
    required_cols = {"index_ID", "SMILES_Prepared"}
    if not required_cols.issubset(df.columns):
        raise ValueError(f"CSV must contain columns: {required_cols}")

    yaml_paths = []

    for _, row in df.iterrows():
        ligand_id = str(row["index_ID"])
        smiles = str(row["SMILES_Prepared"])

        yaml_content = {
            "version": 1,
            "sequences": [
                {"protein": {"id": "A", "sequence": protein_sequence}},
                {"ligand": {"id": "B", "smiles": smiles}},
            ],
            "properties": [
                {"affinity": {"binder": "B"}}
            ],
        }

        yaml_path = os.path.join(output_dir, f"{ligand_id}.yaml")
        with open(yaml_path, "w") as f:
            yaml.dump(yaml_content, f, sort_keys=False)

        yaml_paths.append(yaml_path)

    print(f"✅ Generated {len(yaml_paths)} YAML files in '{output_dir}'")
    return yaml_paths


def run_boltz_prediction(yaml_file, output_dir):
    """
    Runs Boltz-2 prediction on a single YAML file inside its own output folder.
    """
    ligand_name = os.path.splitext(os.path.basename(yaml_file))[0]
    ligand_output_dir = os.path.join(output_dir, ligand_name)
    os.makedirs(ligand_output_dir, exist_ok=True)

    cmd = [
        "boltz",
        "predict",
        os.path.abspath(yaml_file),
        "--use_msa_server"
    ]
    print(f"Running command: {cmd}")

    print(f"▶ Running Boltz-2 for {ligand_name} ...")
    try:
        subprocess.run(cmd, cwd=ligand_output_dir, check=True)
        print(f"✅ Completed: {ligand_name}")
        return True
    except subprocess.CalledProcessError as e:
        print(f"❌ Failed for {ligand_name}: {e}")
        return False

# the main function

In [11]:
if __name__ == "__main__":
    # Example usage
    csv_file = "final_prepared_molecules.csv"          # path to your CSV
    pdb_file = "protein_clean.pdb"          # path to your PDB
    yaml_dir = "all_yaml_files"         # directory for YAML outputs
    boltz_output_dir = "boltz_output" # Directory for Boltz-2 results

    yaml_files = generate_yaml_files(csv_file, pdb_file, output_dir=yaml_dir)
    # Run Boltz-2 predictions for each YAML file
    for yaml_file in yaml_files:
        #print(f"Processing {yaml_file}...")
        run_boltz_prediction(yaml_file, boltz_output_dir)

Extracted protein sequence length: 297 residues
✅ Generated 300 YAML files in 'all_yaml_files'
Running command: ['boltz', 'predict', '/Users/ganeshshahane/Work/TQ_test/all_yaml_files/287.yaml', '--use_msa_server']
▶ Running Boltz-2 for 287 ...


/Users/ganeshshahane/miniconda3/envs/TQ_test/lib/python3.12/site-packages/torch/__init__.py:1617: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/Context.cpp:85.)
  _C._set_float32_matmul_precision(precision)
  0%|          | 0/1 [00:00<?, ?it/s]

MSA server enabled: https://api.colabfold.com
MSA server authentication: no credentials provided
Checking input data.
Processing 1 inputs with 1 threads.
Generating MSA for /Users/ganeshshahane/Work/TQ_test/all_yaml_files/287.yaml with 1 protein entities.
Calling MSA server for target 287 with 1 sequences
MSA server URL: https://api.colabfold.com
MSA pairing strategy: greedy
No authentication provided for MSA server



100%|██████████| 1/1 [00:07<00:00,  7.48s/it]
Using bfloat16 Automatic Mixed Precision (AMP)
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Users/ganeshshahane/miniconda3/envs/TQ_test/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default


Running structure prediction for 1 input.


...

MSA server enabled: https://api.colabfold.com
MSA server authentication: no credentials provided
Checking input data.
Processing 1 inputs with 1 threads.
Generating MSA for /Users/ganeshshahane/Work/TQ_test/all_yaml_files/779.yaml with 1 protein entities.
Calling MSA server for target 779 with 1 sequences
MSA server URL: https://api.colabfold.com
MSA pairing strategy: greedy
No authentication provided for MSA server



100%|██████████| 1/1 [00:08<00:00,  8.80s/it]
Using bfloat16 Automatic Mixed Precision (AMP)
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Users/ganeshshahane/miniconda3/envs/TQ_test/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default


Running structure prediction for 1 input.


/Users/ganeshshahane/miniconda3/envs/TQ_test/lib/python3.12/site-packages/pytorch_lightning/utilities/migration/utils.py:56: The loaded checkpoint was produced with Lightning v2.5.0.post0, which is newer than your current Lightning version: v2.5.0
/Users/ganeshshahane/miniconda3/envs/TQ_test/lib/python3.12/multiprocessing/resource_tracker.py:279: UserWarning: resource_tracker: There appear to be 1 leaked semaphore objects to clean up at shutdown
  warnings.warn('resource_tracker: There appear to be %d '


KeyboardInterrupt: 